# U-Net example
In this notebook we will give a minimal example on how to use U-Net to train/test/predict.

First open up the shell/terminal you used for installation and activate the virtual environment you [installed](https://github.com/CSDGroup/aisegcell#installation). If you want to run this notebook interactively, you will have to manually `pip install jupyterlab` in your shell and check out [the documentation](https://docs.jupyter.org/en/latest/) on how to use it.

## Generate sample data

In [ ]:
# install pooch if necessary
!pip install pooch

In [5]:
import pooch

import pandas as pd

# download sample data
THATCHER = pooch.create(
    path=pooch.os_cache('aisegcell'),
    base_url='https://github.com/CSDGroup/aisegcell/raw/521be0b66d497791d82e75c8211ac62cb31f6a2e/images/',
    registry={
        'bf1.png': (
            '06047311573206ee8c31716ac0e25116a18159e48c1201c7d8b3cdbd398fc2b3'
        ),
        'mask1.png': (
            'dae2999292a623626f900bdb1ed854bf9e33b342ac50dd5633eef7adb5021db1'
        ),
        'bf2.png': (
            '90bf8dc0acb37f39141dd92be79d85f374e3929bb0aa0546e521980749b807d2'
        ),
        'mask2.png': (
            'de488a87cee7c48fe32e8b86833498a25235a56d5ee8ced5d470d31a6c846cd7'
        ),
    }
)

_ = THATCHER.fetch('bf1.png')
_ = THATCHER.fetch('bf2.png')
_ = THATCHER.fetch('mask1.png')
_ = THATCHER.fetch('mask2.png')

# This directory will be different on every machine --> adapt subsequent commands with your path
print(THATCHER.abspath)

/Users/schidani/Library/Caches/aisegcell


In [6]:
# we will use the aisegcell_generate_list command to generate an input file for U-Net with the downloaded files
# we will search all stored images at `THATCHER.abspath` (see above, will be different on every system)
# NOTE: prepending a command with "!" executes the command in the shell/terminal and not with python
# NOTE: use linebreaks ('\') for readability
!aisegcell_generate_list \
--bf "/Users/schidani/Library/Caches/aisegcell/*bf*.png" \
--mask "/Users/schidani/Library/Caches/aisegcell/*mask*.png" \
--out /Users/schidani/Library/Caches/aisegcell/ \
--prefix example

# let's have a look at the input file
input_df = pd.read_csv('/Users/schidani/Library/Caches/aisegcell/example_paths.csv')

input_df

,bf,mask
0,/Users/schidani/Library/Caches/aisegcell/bf1.png,/Users/schidani/Library/Caches/aisegcell/mask1...
1,/Users/schidani/Library/Caches/aisegcell/bf2.png,/Users/schidani/Library/Caches/aisegcell/mask2...


## Training
We have successfully generated the necessary input file and will now move on to train U-Net.

In [7]:
# Have a look at the README.md documentation for detailed explanations of the parameters
!aisegcell_train \
--data /Users/schidani/Library/Caches/aisegcell/example_paths.csv \
--data_val /Users/schidani/Library/Caches/aisegcell/example_paths.csv \
--output_base_dir /Users/schidani/Library/Caches/aisegcell/training \
--devices cpu \
--epochs 2 \
--batch_size 2 \
--shape 128 \
--receptive_field 64 \
--log_frequency 1 \
--bilinear

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type     | Params
----------------------------------
0 | unet | UNet_rec | 10.2 M
----------------------------------
10.2 M    Trainable params
0         Non-trainable params
10.2 M    Total params
40.898    Total estimated model params size (MB)
Epoch 0:  50%|████████        | 1/2 [00:07<00:07,  7.74s/it, loss=1.21, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 1:  50%|████████        | 1/2 [00:07<00:07,  7.74s/it, loss=1.19, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 1: 100%|████████████████| 2/2 [00:15<00:00,  7.74s/it, loss=1.19, v_num=0]


In [9]:
# let's have a look at the output folder (--output_base_dir)
# NOTE: the folder {YYMMDD}_Unet_{ID} will be different for every training
!ls /Users/schidani/Library/Caches/aisegcell/training/230414_Unet_19715/lightning_logs/version_0/

# model weights are saved in the "checkpoints" directory
!ls /Users/schidani/Library/Caches/aisegcell/training/230414_Unet_19715/lightning_logs/version_0/checkpoints/

checkpoints  hparams.yaml metrics.csv
best-f1-epoch=0-step=0.ckpt   best-loss-epoch=1-step=1.ckpt
best-iou-epoch=0-step=0.ckpt  latest-epoch=1-step=1.ckpt


## Testing
We will now use one of the models we generated during training for testing. Testing requires ground truth segmentations for your input images to compare the predicted segmentation masks to. Testing will compute several metrics to quantify the (dis-)similarity between prediction and ground truth.

In [10]:
# Have a look at the README.md documentation for detailed explanations of the parameters
# NOTE: you must generate a trained model (.ckpt file, e.g. training above) for aisegcell_test to work
!aisegcell_test \
--data /Users/schidani/Library/Caches/aisegcell/example_paths.csv \
--model /Users/schidani/Library/Caches/aisegcell/training/230414_Unet_19715/lightning_logs/version_0/checkpoints/latest-epoch\=1-step\=1.ckpt \
--output_base_dir /Users/schidani/Library/Caches/aisegcell/testing \
--suffix mask \
--devices cpu

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Testing: 0it [00:00, ?it/s]/Users/schidani/.pyenv/versions/3.8.6/envs/aisegcell/lib/python3.8/site-packages/torchvision/utils.py:50: UserWarning: range will be deprecated, please use value_range instead.
  warnings.warn(warning)
Testing: 100%|████████████████████████████████████| 2/2 [00:12<00:00,  7.12s/it]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'f1': 0.0,
 'f1_epoch': 0.0,
 'fn': 2.5,
 'fn_epoch': 2.5,
 'fp': 628.5,
 'fp_epoch': 628.5,
 'inaccurate_masks': 0.0,
 'inaccurate_masks_epoch': 0.0,
 'iou': 0.01586497575044632,
 'iou_big': 0.0,
 'iou_big_epoch': 0.0,
 'iou_epoch': 0.01586497575044632,
 'iou_small': 0.01586497575044632,
 'iou_small_epoch': 0.01586497575044632,
 'loss_test': 0.7539274096488953,
 'loss_test_epoch': 0.7539274096488953,
 'merges': 0.0,
 'merges_epoch': 0.0,
 'splits': 0.0,
 'splits_epoc

In [11]:
# let's have a look at the output of aisegcell_test
!ls /Users/schidani/Library/Caches/aisegcell/testing/lightning_logs/version_0/

hparams.yaml metrics.csv  test_masks


## Predicting
Similar to testing we can use trained models for predicting. In contrast to testing, predicting will only generate segmentation masks. Hence, ground truth segmentations are not required and no (dis-)similarity metrics between prediction and ground truth will be computed.

In [12]:
# Have a look at the README.md documentation for detailed explanations of the parameters
# NOTE: you must generate a trained model (.ckpt file, e.g. training above) for aisegcell_predict to work
!aisegcell_predict \
--data /Users/schidani/Library/Caches/aisegcell/example_paths.csv \
--model /Users/schidani/Library/Caches/aisegcell/training/230414_Unet_19715/lightning_logs/version_0/checkpoints/latest-epoch\=1-step\=1.ckpt \
--output_base_dir /Users/schidani/Library/Caches/aisegcell/predicting \
--suffix mask \
--devices cpu

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/schidani/.pyenv/versions/3.8.6/envs/aisegcell/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Predicting: 0it [00:00, ?it/s]/Users/schidani/.pyenv/versions/3.8.6/envs/aisegcell/lib/python3.8/site-packages/torchvision/utils.py:50: UserWarning: range will be deprecated, please use value_range instead.
  warnings.warn(warning)
/Users/schidani/.pyenv/versions/3.8.6/envs/aisegcell/lib/python3.8/site-packages/pytorch_lightning/loops/epoch/prediction_epoch_loop.py:141: UserWarning: predict returned None if it was on purpose, ignore this warning...
  self._warning_cache.w

In [13]:
# let's have a look at the output of aisegcell_predict
# NOTE: no metrics.csv was computed
!ls /Users/schidani/Library/Caches/aisegcell/predicting/lightning_logs/version_0/

# content of predicted_masks directory
!ls /Users/schidani/Library/Caches/aisegcell/predicting/lightning_logs/version_0/predicted_masks/

hparams.yaml    predicted_masks
bf1mask.png bf2mask.png


## (Optional) Clean-up
Finally, let's clean up the `pooch` cache we have cluttered with train/test/predict output.

In [14]:
!rm -r /Users/schidani/Library/Caches/aisegcell/training
!rm -r /Users/schidani/Library/Caches/aisegcell/testing
!rm -r /Users/schidani/Library/Caches/aisegcell/predicting